In [ ]:
!pip install    -q obstore   --upgrade
!pip install    -q duckdb    --upgrade
import sys
sys.exit(0)

In [ ]:
try:
    import duckdb
    duckdb.sql(" force install delta from core_nightly;")
except:
    print("all good")

In [ ]:
ws                    = 'largedata'
lh                    = 'simple'
schema                = 'test'
compaction_threshold  =  150
sql_folder            = 'https://github.com/djouallah/fabric_demo/raw/refs/heads/main/transformation/'

<mark>**Core Logic**</mark>

In [ ]:
import duckdb
import requests
import os
import sys
import importlib.util
from   deltalake import DeltaTable, write_deltalake
from   typing import List, Tuple, Union, Any, Optional, Callable, Dict
from   string import Template
class Tasksql:
    """
    Simplified Lakehouse task runner supporting:
      - ('script_name', (args,))          → runs script_name.py → script_name(*args)
      - ('table_name', 'mode', {params})  → runs table_name.sql with params, writes to Delta
    """

    def __init__(self, workspace: str, lakehouse_name: str, schema: str, sql_folder: str, compaction_threshold: int = 10):
        self.workspace = workspace
        self.lakehouse_name = lakehouse_name
        self.schema = schema
        self.sql_folder = sql_folder.strip()
        self.compaction_threshold = compaction_threshold
        self.table_base_url = f'abfss://{self.workspace}@onelake.dfs.fabric.microsoft.com/{self.lakehouse_name}.Lakehouse/Tables/'
        self.con = duckdb.connect()
        self._attach_lakehouse()

    @classmethod
    def connect(cls, workspace: str, lakehouse_name: str, schema: str, sql_folder: str, compaction_threshold: int = 10):
        print("Connecting to Lakehouse...")
        return cls(workspace, lakehouse_name, schema, sql_folder.strip(), compaction_threshold)

    def _get_storage_token(self):
        return os.environ.get("AZURE_STORAGE_TOKEN", "PLACEHOLDER_TOKEN_TOKEN_NOT_AVAILABLE")

    def _create_onelake_secret(self):
        token = self._get_storage_token()
        if token != "PLACEHOLDER_TOKEN_TOKEN_NOT_AVAILABLE":
            self.con.sql(f"CREATE OR REPLACE SECRET onelake (TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}')")
        else:
            print("Please login to Azure CLI")
            self.con.sql("CREATE OR REPLACE PERSISTENT SECRET onelake (TYPE azure, PROVIDER credential_chain, CHAIN 'cli', ACCOUNT_NAME 'onelake')")

    def _attach_lakehouse(self):
        self._create_onelake_secret()
        try:
            list_tables_query = f"""
                SELECT DISTINCT(split_part(file, '_delta_log', 1)) as tables
                FROM glob ("abfss://{self.workspace}@onelake.dfs.fabric.microsoft.com/{self.lakehouse_name}.Lakehouse/Tables/*/*/_delta_log/*.json")
            """
            list_tables_df = self.con.sql(list_tables_query).df()
            list_tables = list_tables_df['tables'].tolist() if not list_tables_df.empty else []

            if not list_tables:
                print(f"No Delta tables found in {self.lakehouse_name}.Lakehouse/Tables.")
                return

            print(f"Found {len(list_tables)} Delta tables. Attaching as views...")

            for table_path in list_tables:
                parts = table_path.strip("/").split("/")
                if len(parts) >= 2:
                    potential_schema = parts[-2]
                    table = parts[-1]
                    if potential_schema == self.schema:
                        try:
                            self.con.sql(f"""
                                CREATE OR REPLACE VIEW {table}
                                AS SELECT * FROM delta_scan('{self.table_base_url}{self.schema}/{table}');
                            """)
                        except Exception as e:
                            print(f"Error creating view for table {table}: {e}")
            print("\nAttached tables (views) in DuckDB:")
            self.con.sql("SELECT name FROM (SHOW ALL TABLES) WHERE database='memory'").show()
        except Exception as e:
            print(f"Error attaching lakehouse: {e}")

    def _normalize_table_name(self, name: str) -> str:
        """
        Extract base table name by taking the part before the first double underscore '__'.
        If no underscore, return the name as-is.
        
        Examples:
            'sales__update' → 'sales'
            'fact__daily_v2' → 'fact'
            'events' → 'events'
        """
        if '__' in name:
            return name.split('__', 1)[0]
        return name

    def _read_sql_file(self, table_name: str, params: Optional[Dict] = None) -> Optional[str]:
        is_url = self.sql_folder.startswith("http")
        if is_url:
            url = f"{self.sql_folder.rstrip('/')}/{table_name}.sql".strip()
            try:
                resp = requests.get(url)
                resp.raise_for_status()
                content = resp.text
            except Exception as e:
                print(f"Failed to fetch SQL from {url}: {e}")
                return None
        else:
            path = os.path.join(self.sql_folder, f"{table_name}.sql")
            try:
                with open(path, 'r') as f:
                    content = f.read()
            except Exception as e:
                print(f"Failed to read SQL file {path}: {e}")
                return None

        if not content.strip():
            print(f"SQL file is empty: {table_name}.sql")
            return None

        # Merge system + user params
        full_params = {
            'ws': self.workspace,
            'lh': self.lakehouse_name,
            'schema': self.schema
        }
        if params:
            full_params.update(params)

        # Use string.Template ($ws, ${run_date}) — safe with DuckDB {}
        try:
            template = Template(content)
            content = template.substitute(full_params)
        except KeyError as e:
            print(f"Missing parameter in SQL file: ${e}")
            return None
        except Exception as e:
            print(f"Error during SQL template substitution: {e}")
            return None

        return content

    def _load_py_function(self, name: str) -> Optional[Callable]:
        is_url = self.sql_folder.startswith("http")
        try:
            if is_url:
                url = f"{self.sql_folder.rstrip('/')}/{name}.py".strip()
                resp = requests.get(url)
                resp.raise_for_status()
                code = resp.text
                namespace = {}
                exec(code, namespace)
                func = namespace.get(name)
                return func if callable(func) else None
            else:
                path = os.path.join(self.sql_folder, f"{name}.py")
                if not os.path.isfile(path):
                    print(f"Python file not found: {path}")
                    return None
                spec = importlib.util.spec_from_file_location(name, path)
                mod = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(mod)
                func = getattr(mod, name, None)
                return func if callable(func) else None
        except Exception as e:
            print(f"Error loading Python function '{name}': {e}")
            return None

    def _run_py_task(self, name: str, args: tuple) -> int:
        func = self._load_py_function(name)
        if not func:
            return 0
        try:
            print(f"Running Python task: {name}{args}")
            result = func(*args)
            print(f"✅ Python task '{name}' completed.")
            return result
        except Exception as e:
            print(f"❌ Error in Python task '{name}': {e}")
            return 0

    def _run_sql_task(self, table: str, mode: str, params: Optional[Dict] = None) -> int:
        allowed_modes = {'overwrite', 'append', 'ignore'}
        if mode not in allowed_modes:
            print(f"Invalid mode '{mode}'. Use: {allowed_modes}")
            return 0

        sql = self._read_sql_file(table, params)  # loads table.sql or table_anything.sql as-is
        if sql is None:
            return 0

        normalized_table = self._normalize_table_name(table)
        path = f"{self.table_base_url}{self.schema}/{normalized_table}"

        try:
            if mode == 'overwrite':
                self.con.sql(f"DROP VIEW IF EXISTS {normalized_table}")
                df = self.con.sql(sql).arrow()
                write_deltalake(
                    path, df, mode='overwrite',
                    max_rows_per_file=8_000_000,
                    max_rows_per_group=8_000_000,
                    min_rows_per_group=8_000_000,
                    engine='pyarrow'
                )
                self.con.sql(f"CREATE OR REPLACE VIEW {normalized_table} AS SELECT * FROM delta_scan('{path}')")
                dt = DeltaTable(path)
                dt.vacuum(retention_hours=0, dry_run=False, enforce_retention_duration=False)
                dt.cleanup_metadata()

            elif mode == 'append':
                df = self.con.sql(sql).arrow()
                write_deltalake(
                    path, df, mode='append',
                    max_rows_per_file=8_000_000,
                    max_rows_per_group=8_000_000,
                    min_rows_per_group=8_000_000,
                    engine='pyarrow'
                )
                self.con.sql(f"CREATE OR REPLACE VIEW {normalized_table} AS SELECT * FROM delta_scan('{path}')")
                dt = DeltaTable(path)
                if len(dt.files()) > self.compaction_threshold:
                    print(f"Compacting {normalized_table} (files: {len(dt.files())})")
                    dt.optimize.compact()
                    dt.vacuum(dry_run=False)
                    dt.cleanup_metadata()

            elif mode == 'ignore':
                try:
                    DeltaTable(path)
                    print(f"Table {normalized_table} exists. Skipping (mode='ignore').")
                except Exception:
                    print(f"{normalized_table} doesn't exist. Creating in overwrite mode.")
                    self.con.sql(f"DROP VIEW IF EXISTS {normalized_table}")
                    df = self.con.sql(sql).arrow()
                    write_deltalake(
                        path, df, mode='overwrite',
                        max_rows_per_file=8_000_000,
                        max_rows_per_group=8_000_000,
                        min_rows_per_group=8_000_000,
                        engine='pyarrow'
                    )
                    self.con.sql(f"CREATE OR REPLACE VIEW {normalized_table} AS SELECT * FROM delta_scan('{path}')")
                    dt = DeltaTable(path)
                    dt.vacuum(dry_run=False)
                    dt.cleanup_metadata()

            print(f"✅ SQL task '{table}' → table '{normalized_table}' ({mode}) completed.")
            return 1

        except Exception as e:
            print(f"❌ Error in SQL task '{table}' (writing to '{normalized_table}'): {e}")
            return 0

    def run_pipeline(self, tasks: List[Union[Tuple[str, tuple], Tuple[str, str, Dict]]]) -> bool:
        """
        Run tasks with simple syntax:
          - ('download', (url_list, path_list, depth))
          - ('staging_table', 'overwrite', {'run_date': '2024-06-01'})
        """
        for i, task in enumerate(tasks):
            print(f"\n--- Running Task {i+1}: {task[0]} ---")
            name = task[0]

            if len(task) == 2:
                # Python task: ('name', (args,))
                args = task[1]
                if not isinstance(args, (tuple, list)):
                    args = (args,)
                result = self._run_py_task(name, tuple(args))
            elif len(task) == 3:
                # SQL write task: ('table', 'mode', {params})
                mode, params = task[1], task[2]
                if not isinstance(params, dict):
                    print(f"❌ Expected dict as 3rd item in SQL task, got {type(params)}")
                    return False
                result = self._run_sql_task(name, mode, params)
            else:
                print(f"❌ Invalid task format: {task}")
                return False

            if result != 1:
                print(f"❌ Task {i+1} failed. Stopping.")
                return False

        print("\n✅ All tasks completed successfully.")
        return True

    def get_connection(self):
        return self.con

    def close(self):
        if self.con:
            self.con.close()
            print("DuckDB connection closed.")

In [ ]:
%%time
con = Tasksql.connect( ws,lh,schema, sql_folder, compaction_threshold  )

In [ ]:
nightly =[
              ('scraping', (["https://nemweb.com.au/Reports/Current/Daily_Reports/"],["Reports/Current/Daily_Reports/"],60,ws,lh,6)),
              ('price','append',{'ws': ws,'lh':lh}),
              ('scada','append',{'ws': ws,'lh':lh}),
              ('download_excel',("raw/", ws,lh)),
              ('duid','overwrite',{'ws': ws,'lh':lh}),
              ('calendar','ignore',{}),
              ('mstdatetime','ignore',{}),
              ('summary__backfill','overwrite',{})
                     ]

intraday = [
              ('scraping', (["http://nemweb.com.au/Reports/Current/DispatchIS_Reports/", "http://nemweb.com.au/Reports/Current/Dispatch_SCADA/" ],
                            ["Reports/Current/DispatchIS_Reports/","Reports/Current/Dispatch_SCADA/"],
                             288, ws,lh,6)),
              ('price_today','append',{'ws': ws,'lh':lh}),
              ('scada_today','append',{'ws': ws,'lh':lh}),
              ('duid','ignore',{'ws': ws,'lh':lh}),
              ('summary__incremental', 'append',{})            
                    ]
history_download = [
               ('scraping',
                          (["https://github.com/djouallah/fabric_demo/tree/main/data/archive/2025",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2024",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2023",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2022",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2021",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2020",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2019",
                            "https://github.com/djouallah/fabric_demo/tree/main/data/archive/2018"
                          ],
                         ["Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/",
                          "Reports/Current/Daily_Reports/"
                         ]
                         ,7,ws,lh,6))
                      ]
history_process = [
              ('price','append',{'ws': ws,'lh':lh}),
              ('scada','append',{'ws': ws,'lh':lh}),
              ('summary__backfill_archive','append',{})
                   ]

In [ ]:
%%time
con.run_pipeline(nightly)

In [ ]:
%%time
con.run_pipeline(intraday)

In [ ]:
%%time
## you can turn it off when Historical data is fully loaded
con.run_pipeline(history_download)
con.run_pipeline(history_process)